In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random


## PHẦN 1: KHAI BÁO BIẾN VÀ THƯ VIỆN

# Mục tiêu:
- Import các thư viện cần thiết
- Khai báo URL cơ sở và headers để giả lập trình duyệt

# Giải thích:
- `BASE_URL`: Template URL với placeholder `{}` cho số trang
- `HEADERS`: Giả lập User-Agent của Chrome để tránh bị website chặn
- Các thư viện: requests (gử HTTP), BeautifulSoup (parse HTML), pandas (xử lý data), time & random (tạm dừng)


In [ ]:
BASE_URL = "https://homedy.com/ban-nha-dat/p{}"  
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

## PHẦN 2: KHỞI TẠO BIẾN VÀ DỮ LIỆU


# Mục tiêu:
- Khởi tạo các biến cần thiết cho quá trình crawl
- Thiết lập giới hạn dữ liệu cần thu thập

# Giải thích:
- `data`: List rỗng để lưu trữ tất cả các bản ghi thu thập được
- `max_data`: Số lượng mẫu tối đa muốn thu thập (2000 theo yêu cầu)
- `page`: Biến đếm trang, bắt đầu từ trang 1


In [ ]:
data = []
max_data = 2000
page = 1

## PHẦN 3: VÒNG LẶP CRAWL DỮ LIỆU TỪ NHIỀU TRANG

# Mục tiêu:
- Thực hiện vòng lặp qua tất cả các trang cho đến khi đủ 2000 mẫu dữ liệu
- Xử lý lỗi và kiểm tra điều kiện dừng

# Logic thực hiện:
1. Tạo URL cho từng trang (trang 1 có URL đặc biệt)
2. Gửi request và kiểm tra phản hồi
3. Phát hiện trang cuối cùng bằng cách kiểm tra redirect
4. Parse HTML và tìm tất cả các item chứa thông tin bất động sản

In [ ]:
while len(data) < max_data:
    if page == 1:
        url = "https://homedy.com/ban-nha-dat"
    else:
        url = BASE_URL.format(page)
    
    print(f"Đang crawl trang {page}: {url}")
    
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        
        if response.url != url and "ban-nha-dat" not in response.url:
            print("Đã đến trang cuối cùng. Dừng crawl.")
            break
            
    except requests.exceptions.RequestException as e:
        print(f"Lỗi khi gửi yêu cầu đến trang {page}: {e}")
        break

    soup = BeautifulSoup(response.content, 'html.parser')
    list_items = soup.find_all('div', class_='product-content')
    
    if not list_items:
        print("Không tìm thấy tin đăng nào. Kết thúc crawl.")
        break


 ## PHẦN 4: TRÍCH XUẤT VÀ XỬ LÝ DỮ LIỆU

  # Mục tiêu:
  - Trích xuất thông tin từ mỗi item (tin đăng)
  - Lọc và làm sạch dữ liệu
  - Thêm vào danh sách tổng hợp
    
  # Các trường dữ liệu được trích xuất:
  - Địa chỉ (address): từ thẻ <li class="address">
  - Diện tích (acreage): từ thẻ <span class="acreage"> 
  - Giá (price): từ thẻ <span class="price">
  - Trang (page): số trang để theo dõi


In [ ]:
items_count = 0
    for item in list_items:
        try:
            price_element = item.find('span', class_='price')
            price = price_element.get_text(strip=True) if price_element else "Không rõ"
            
            acreage_element = item.find('span', class_='acreage')
            acreage = acreage_element.get_text(strip=True) if acreage_element else "Không rõ"
            
            address_element = item.find('li', class_='address')
            address = address_element.get_text(strip=True) if address_element else "Không rõ"
            
            if price != "Không rõ" or acreage != "Không rõ" or address != "Không rõ":
                data.append({
                    'Địa chỉ': address,
                    'Diện tích': acreage,
                    'Giá': price,
                    'Trang': page
                })
                items_count += 1
                
            if len(data) >= max_data:
                break
                
        except Exception as e:
            print(f"Lỗi khi trích xuất dữ liệu từ một item: {e}")
            continue
    
    print(f"Đã thu thập {items_count} mẫu từ trang {page}. Tổng cộng: {len(data)}/{max_data}")
    
    page += 1
    
    sleep_time = random.uniform(2, 5)
    print(f"Tạm dừng {sleep_time:.2f} giây...")
    time.sleep(sleep_time)

Đã thu thập 2 mẫu từ trang 112. Tổng cộng: 2000/2000
Tạm dừng 3.16 giây...



## PHẦN 5: LƯU TRỮ VÀ XUẤT DỮ LIỆU

# Mục tiêu:
- Chuyển đổi dữ liệu thành DataFrame
- Hiển thị thống kê và preview dữ liệu
- Lưu thành file CSV để sử dụng sau này

# Đặc điểm:
- File CSV được lưu với encoding UTF-8 với BOM để hỗ trợ tiếng Việt trong Excel
- Tên file chứa timestamp để tránh ghi đè
- Hiển thị 5 mẫu đầu tiên để kiểm tra


In [ ]:
if data:
    df = pd.DataFrame(data)
    print(f"\nĐã thu thập thành công {len(df)} mẫu dữ liệu!")
    print("\n5 mẫu đầu tiên:")
    print(df.head())
    
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    filename = f'vietnam_housing_data_{timestamp}.csv'
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\nĐã lưu dữ liệu vào file '{filename}'")
else:
    print("Không tìm thấy dữ liệu nào để lưu!")


Đã thu thập thành công 2000 mẫu dữ liệu!

5 mẫu đầu tiên:
                              Địa chỉ    Diện tích           Giá  Trang
0           Văn Giang,  tỉnh Hưng Yên  28 - 112 m2  1,9 - 6,5 Tỷ      1
1    Quận Bình Thạnh,  TP Hồ Chí Minh        50 m2          2 Tỷ      1
2            Quận 10,  TP Hồ Chí Minh        63 m2     970 Triệu      1
3  quận Ninh Kiều,  thành phố Cần Thơ        70 m2        1,5 Tỷ      1
4      Thành phố Quy Nhơn,  Bình Định   50 - 96 m2  2,4 - 7,9 Tỷ      1

Đã lưu dữ liệu vào file 'vietnam_housing_data_20251028_205210.csv'
